[WBF inference github link](https://github.com/ZFTurbo/Weighted-Boxes-Fusion)

In [318]:
# !pip install ensemble-boxes

In [211]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

# for WBF ensemble
from ensemble_boxes import *
import cv2
from tqdm import tqdm

## Load model

In [309]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
#cfg = Config.fromfile('./configs/cascade_rpn/crpn_faster_rcnn_r50_caffe_fpn_1x_coco.py')
#cfg = Config.fromfile('./configs/trash/faster_rcnn/faster_rcnn_r50_fpn_1x_trash.py')
cfg = Config.fromfile('./configs/vfnet/vfnet_x101_64x4d_fpn_mstrain_2x_coco.py')

PREFIX = '../../input/data/'

epoch = 49

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 8

cfg.seed=2020
cfg.gpu_ids = [0]

cfg.work_dir = './work_dirs/vfnet_x101_64x4d_fpn_mstrain_2x_coco'
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'
#cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [310]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [311]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-13 17:53:27,423 - mmdet - INFO - load model from: open-mmlab://resnext101_64x4d
2021-05-13 17:53:27,424 - mmdet - INFO - Use load_from_openmmlab loader


Use load_from_local loader


## Create outputs to ensemble

In [308]:
output1 = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 10.0 task/s, elapsed: 83s, ETA:     0s

In [317]:
########################################################
#### Go to the first section and load another model ####
########################################################

In [312]:
# Load model 2 before implement this cell
output2 = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 10.1 task/s, elapsed: 83s, ETA:     0s

## WBF (ensemble)

- output1 from Model 1
- output2 from Model 2

In [313]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()
img_size = 512.

weights = [2, 1] # ensemble weights for model 1 and model 2
iou_thr = 0.6
skip_box_thr = 0.0001

for idx in tqdm(range(len(output1))):
    
    boxes_list1, boxes_list2, boxes_list = [], [], []
    scores_list1, scores_list2, scores_list = [], [], []
    labels_list1, labels_list2, labels_list = [], [], []
    
    # model 1
    for label, boxes_in_label in enumerate(output1[idx]):
        for box_and_score in boxes_in_label:
            scores_list1.append(box_and_score[4])
            boxes_list1.append(box_and_score[:4] / img_size)
            labels_list1.append(label)
    
    # model 2
    for label, boxes_in_label in enumerate(output2[idx]):
        for box_and_score in boxes_in_label:
            scores_list2.append(box_and_score[4])
            boxes_list2.append(box_and_score[:4] / img_size)
            labels_list2.append(label)
    
    boxes_list = [boxes_list1, boxes_list2]
    scores_list = [scores_list1, scores_list2]
    labels_list = [labels_list1, labels_list2]
    
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=idx))[0]
    
    for i, box in enumerate(boxes):
        prediction_string += str(int(labels[i])) + ' ' + str(scores[i])[:11] + ' ' + str(box[0]*img_size)[:9] + ' '  + str(box[1]*img_size)[:9] + ' '  + str(box[2]*img_size)[:9] + ' '  + str(box[3]*img_size)[:9] + ' '
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])

  1%|          | 8/837 [00:00<00:11, 73.45it/s]

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


100%|██████████| 837/837 [00:10<00:00, 78.40it/s] 


In [315]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_wbf_ensemble.csv'), index=None)
submission.head()

,PredictionString,image_id
0,8 0.988392353 173.44706 235.69149 462.39956 47...,batch_01_vt/0021.jpg
1,8 0.965389311 158.91023 57.217319 471.61865 25...,batch_01_vt/0028.jpg
2,1 0.81283784 69.730636 169.58737 376.14859 385...,batch_01_vt/0031.jpg
3,8 0.982105791 30.924922 124.46896 388.22097 37...,batch_01_vt/0032.jpg
4,8 0.981500625 149.44270 0.0 354.38168 214.2527...,batch_01_vt/0070.jpg


## Default NMS

In [245]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(o[2]) + ' ' + str(o[3]) + ' '
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [246]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

,PredictionString,image_id
0,0 0.069298856 165.7254 74.91447 255.95776 165....,batch_01_vt/0021.jpg
1,1 0.34880495 42.91632 0.117673874 103.32407 60...,batch_01_vt/0028.jpg
2,0 0.14965408 234.38148 462.2145 302.18164 511....,batch_01_vt/0031.jpg
3,0 0.06412559 301.05057 160.59392 363.67383 225...,batch_01_vt/0032.jpg
4,1 0.5320658 389.231 266.6002 497.96707 342.480...,batch_01_vt/0070.jpg


## WBF (single model)

In [299]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()
img_size = 512.

iou_thr = 0.6
skip_box_thr = 0.0001

for idx, img in enumerate(tqdm(output)):
    
    boxes_list = []
    scores_list = []
    labels_list = []
    
    for label, boxes_in_label in enumerate(img):
        for box_and_score in boxes_in_label:
            scores_list.append(box_and_score[4])
            boxes_list.append(box_and_score[:4] / img_size)
            labels_list.append(label)
            
    boxes, scores, labels = weighted_boxes_fusion([boxes_list], [scores_list], [labels_list], weights=None, iou_thr=iou_thr, skip_box_thr=0.0)
    
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=idx))[0]
    
    for i, box in enumerate(boxes):
        prediction_string += str(int(labels[i])) + ' ' + str(scores[i])[:11] + ' ' + str(box[0]*img_size)[:9] + ' '  + str(box[1]*img_size)[:9] + ' '  + str(box[2]*img_size)[:9] + ' '  + str(box[3]*img_size)[:9] + ' '
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])

  4%|▎         | 31/837 [00:00<00:02, 309.21it/s]

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


100%|██████████| 837/837 [00:03<00:00, 265.60it/s]


In [ ]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()